In [130]:
import pandas
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [131]:
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.decomposition import PCA  # to apply PCA

In [132]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.spatial


module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [134]:


def processText(column,n):#to remove stop words,to convert into lowercase and root word
    import re

    corpus = []
    for i in range(n):
        review = re.sub('[^a-zA-Z]', ' ', column[i])
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        all_stopwords = stopwords.words('english')
        all_stopwords.remove('not')
        review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
        review = ' '.join(review)
        corpus.append(review)
    return corpus


In [135]:
def embed(input):
  return model(input)

In [136]:


df = pandas.read_csv('./'+'ADBE_combine.csv')


df=df.dropna()
df = df[~(df['datetime'] >= '2023-01-01')]#dropping rows of year=2023

df=df.reset_index(drop=True)


print(df.columns)

titleColumn=df['title']
summaryColumn=df['summary']
dateColumn=df['datetime']
titleColumn=list(titleColumn)
summaryColumn=list(summaryColumn)

corpus=processText(titleColumn,len(titleColumn))#preprocessing title column
df['title']=corpus

summaryCorpus=processText(summaryColumn,len(summaryColumn))#preprocessing summary column
df['summary']=summaryCorpus


df = df.astype({'title':'string'})
df = df.astype({'summary':'string'})

print(df.tail(5))

Index(['datetime', 'close', 'open', 'low', 'high', 'volume', 'y_actual', 'evm',
       'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
       'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',
       'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
       'Free Cash Flow', 'Total Assets',
       'Total Liabilities Net Minority Interest',
       'Total Equity Gross Minority Interest', 'Total Capitalization',
       'summary', 'title'],
      dtype='object')
       datetime       close        open         low        high     volume  \
969  2022-12-22  336.519989  336.010010  330.940002  339.880005  2319300.0   
970  2022-12-23  338.450012  335.000000  333.290009  338.920013  1629800.0   
971  2022-12-27  335.089996  338.000000  332.459991  338.160004  1464300.0   
972  2022-12-28  328.329987  335.059998  327.190002  338.500000  1672100.0   
973  2022-12-29  337.579987  330.750000  329.760010  339.500000  1793100.0   

     y

In [137]:
df.shape

(974, 29)

In [138]:
titleColumn=df['title']
titleColumn[0]

# df=df.fillna(0)

# df=df.dropna()

'need know flickr pro adob discount petapixel th annual adob home tour pauma valley valley roadrunn adob lightroom cc io ad shortcut appl promis verg adob stock climb motley fool g crowd announc best softwar award googl busi wire u holiday shopper spend record billion onlin adob fiji time free altern major adob cc program diy photographi adob acquir allegorithm uktn uk technolog news adob acquir allegorithm maker substanc textur tool techcrunch adob acquir textur firm allegorithm zdnet fill hole adob propos foreground awar imag inpaint sync adob head digit experi busi brad rencher quit tech observ news adob acquir softwar compani allegorithm architosh docusign face new e signatur competit adob pcmag adob photoshop sketch illustr draw app support macrumor luminar review viabl altern adob lightroom macworld brad rencher exit adob key market cloud leader adexchang dropbox acquir hellosign goe docusign adob busi insid mg motor partner adob launch digit focus market exchang media adob plan 

In [139]:
summaryColumn=df['summary']
summaryColumn[0]

'samsung new flagship lineup go get exclus version adob premier rush nadob premier rush allow video edit goth galaxi powerhous camera depart partnership make perfect sens nsamsung confirm unpack event today galaxi receiv exclus version adob premier rush app later year nit said app would optim samsung adob premier rush current avail io devic nwhether not partnership lead discount premier rush galaxi owner remain seen well'

In [140]:
df.head(5)

,datetime,close,open,low,high,volume,y_actual,evm,force_index,rsi,...,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Free Cash Flow,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,summary,title
0,2019-02-20,258.100006,258.130005,255.919998,260.989990,2484800.0,0.0,189.291676,3.834048e+07,67.241092,...,4421314.0,-455085.0,-2946044.0,4087333.0,20762400.0,10232245.0,10530155.0,11519079.0,samsung new flagship lineup go get exclus vers...,need know flickr pro adob discount petapixel t...
1,2019-02-21,257.760010,257.299988,255.119995,259.350006,2607400.0,1.0,131.802598,2.591756e+07,66.485336,...,4421314.0,-455085.0,-2946044.0,4087333.0,20762400.0,10232245.0,10530155.0,11519079.0,samsung new flagship lineup go get exclus vers...,galaxi get exclus version adob premier rush sa...
2,2019-02-22,258.899994,257.609985,257.609985,260.239990,2006000.0,0.0,117.732791,2.310910e+07,67.792451,...,4421314.0,-455085.0,-2946044.0,4087333.0,20762400.0,10232245.0,10530155.0,11519079.0,adob releas updat adob premier pro via creativ...,adob issu premier pro fix bug caus blown macrumor
3,2019-02-25,258.619995,261.500000,258.179993,262.980011,1815800.0,1.0,80.894933,7.045285e+06,67.100230,...,4421314.0,-455085.0,-2946044.0,4087333.0,20762400.0,10232245.0,10530155.0,11519079.0,golf pga tour announc new multi year market pa...,get crash cours adob creativ cloud deal cult m...
4,2019-02-26,261.220001,258.000000,256.369995,262.579987,2538900.0,1.0,21.418314,1.624894e+07,70.148368,...,4421314.0,-455085.0,-2946044.0,4087333.0,20762400.0,10232245.0,10530155.0,11519079.0,adob start send email enterpris client immin r...,adob send email retir shockwav april th bleepi...


In [141]:
df.tail(5)

,datetime,close,open,low,high,volume,y_actual,evm,force_index,rsi,...,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Free Cash Flow,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,summary,title
969,2022-12-22,336.519989,336.010010,330.940002,339.880005,2319300.0,1.0,-79.013019,-1.161972e+07,52.496992,...,7838000.0,-570000.0,-6825000.0,7396000.0,27165000.0,13114000.0,14051000.0,17680000.0,year saw massiv round job cut hire freez tech ...,big tech layoff microsoft meta twitter adob in...
970,2022-12-23,338.450012,335.000000,333.290009,338.920013,1629800.0,0.0,25.302410,7.105954e+06,53.580228,...,7838000.0,-570000.0,-6825000.0,7396000.0,27165000.0,13114000.0,14051000.0,17680000.0,first heard adob road fellow auto journalist t...,review adob road race theme wine collect jalopnik
971,2022-12-27,335.089996,338.000000,332.459991,338.160004,1464300.0,0.0,111.689543,5.769346e+06,51.383437,...,7838000.0,-570000.0,-6825000.0,7396000.0,27165000.0,13114000.0,14051000.0,17680000.0,purchas figma adob combin power creativ tool u...,content creator high demand holiday season unc...
972,2022-12-28,328.329987,335.059998,327.190002,338.500000,1672100.0,1.0,41.699158,2.758955e+06,47.191288,...,7838000.0,-570000.0,-6825000.0,7396000.0,27165000.0,13114000.0,14051000.0,17680000.0,purchas figma adob combin power creativ tool u...,purchas figma make sens adob nasdaq
973,2022-12-29,337.579987,330.750000,329.760010,339.500000,1793100.0,0.0,27.779755,8.965500e+06,52.858810,...,7838000.0,-570000.0,-6825000.0,7396000.0,27165000.0,13114000.0,14051000.0,17680000.0,tl dr get adob creativ cloud beginn advanc bun...,becom expert adob creativ cloud program mashab...


In [142]:
df.columns

Index(['datetime', 'close', 'open', 'low', 'high', 'volume', 'y_actual', 'evm',
       'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
       'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',
       'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
       'Free Cash Flow', 'Total Assets',
       'Total Liabilities Net Minority Interest',
       'Total Equity Gross Minority Interest', 'Total Capitalization',
       'summary', 'title'],
      dtype='object')

In [145]:
df['y_actual']

0      0.0
1      1.0
2      0.0
3      1.0
4      1.0
      ... 
969    1.0
970    0.0
971    0.0
972    1.0
973    0.0
Name: y_actual, Length: 974, dtype: float64

In [146]:
y=df['y_actual']
# y=np.array(y)
y

0      0.0
1      1.0
2      0.0
3      1.0
4      1.0
      ... 
969    1.0
970    0.0
971    0.0
972    1.0
973    0.0
Name: y_actual, Length: 974, dtype: float64

In [147]:
column_names=["y_actual"]

# Create DataFrame by assigning column names
y_df=pd.DataFrame(y, columns=column_names)

In [148]:
y_df

,y_actual
0,0.0
1,1.0
2,0.0
3,1.0
4,1.0
...,...
969,1.0
970,0.0
971,0.0
972,1.0


In [149]:
# headers = ['y_lables']
# y_df = pd.DataFrame(y, columns=headers)
y_df['datetime']=df['datetime']

In [150]:
y_df.head(5)

,y_actual,datetime
0,0.0,2019-02-20
1,1.0,2019-02-21
2,0.0,2019-02-22
3,1.0,2019-02-25
4,1.0,2019-02-26


In [151]:
y_df.shape

(974, 2)

In [152]:
y_df.tail(5)

,y_actual,datetime
969,1.0,2022-12-22
970,0.0,2022-12-23
971,0.0,2022-12-27
972,1.0,2022-12-28
973,0.0,2022-12-29


In [153]:
# df_title=df[['datetime','title']]
df=df.drop(['datetime','title','summary','y_actual'],axis='columns')#dropping the date column


In [154]:
scalar = StandardScaler()
scaled_df = scalar.fit_transform(df) #scaling the csv data

df=scaled_df


In [155]:
df=pd.DataFrame(df)
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-1.395093,-1.393648,-1.377238,-1.402761,-0.253108,0.186944,0.341627,1.059945,0.240676,-0.042876,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
1,-1.398045,-1.400842,-1.384250,-1.416864,-0.180434,0.134354,0.265287,1.001807,-0.016483,-0.089443,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
2,-1.388148,-1.398155,-1.362427,-1.409211,-0.536928,0.121484,0.248028,1.102359,0.229683,-0.104037,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
3,-1.390579,-1.364437,-1.357431,-1.385649,-0.649673,0.087785,0.149314,1.049109,0.440741,-0.125033,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
4,-1.368009,-1.394775,-1.373294,-1.389089,-0.221039,0.033378,0.205872,1.283591,0.422234,-0.093246,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,-0.714332,-0.718593,-0.719750,-0.724378,-0.351212,-0.058495,0.034614,-0.074267,-0.136186,-0.170713,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
970,-0.697578,-0.727348,-0.699154,-0.732633,-0.759929,0.036931,0.149686,0.009063,-0.000362,-0.145667,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
971,-0.726746,-0.701344,-0.706429,-0.739169,-0.858032,0.115955,0.141473,-0.159929,-0.279276,-0.202053,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
972,-0.785429,-0.726828,-0.752616,-0.736245,-0.734854,0.051930,0.122974,-0.482416,-0.941740,-0.375894,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898


In [156]:
df.shape

(974, 25)

In [157]:
from textblob import TextBlob

In [158]:
titleSentiment=[]

In [159]:
for content in titleColumn:
    blob = TextBlob(content)
    sentiment=blob.sentiment.polarity
    if sentiment<0:
      titleSentiment.append(0)
    else:
      titleSentiment.append(1)

In [163]:
summarySentiment=[]

In [164]:
for content in summaryColumn:
    blob = TextBlob(content)
    sentiment=blob.sentiment.polarity
    if sentiment<0:
      summarySentiment.append(0)
    else:
      summarySentiment.append(1)

In [166]:
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-1.395093,-1.393648,-1.377238,-1.402761,-0.253108,0.186944,0.341627,1.059945,0.240676,-0.042876,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
1,-1.398045,-1.400842,-1.384250,-1.416864,-0.180434,0.134354,0.265287,1.001807,-0.016483,-0.089443,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
2,-1.388148,-1.398155,-1.362427,-1.409211,-0.536928,0.121484,0.248028,1.102359,0.229683,-0.104037,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
3,-1.390579,-1.364437,-1.357431,-1.385649,-0.649673,0.087785,0.149314,1.049109,0.440741,-0.125033,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
4,-1.368009,-1.394775,-1.373294,-1.389089,-0.221039,0.033378,0.205872,1.283591,0.422234,-0.093246,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,-0.714332,-0.718593,-0.719750,-0.724378,-0.351212,-0.058495,0.034614,-0.074267,-0.136186,-0.170713,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
970,-0.697578,-0.727348,-0.699154,-0.732633,-0.759929,0.036931,0.149686,0.009063,-0.000362,-0.145667,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
971,-0.726746,-0.701344,-0.706429,-0.739169,-0.858032,0.115955,0.141473,-0.159929,-0.279276,-0.202053,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
972,-0.785429,-0.726828,-0.752616,-0.736245,-0.734854,0.051930,0.122974,-0.482416,-0.941740,-0.375894,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898


In [167]:

text_vector=[]# will contain vector representation of titleColumn
summary_vector=[]

In [168]:
for text in titleColumn:
    sentence = text
    message_embeddings = embed([sentence])
    temp=np.array(message_embeddings[0])
    temp=temp.T
    text_vector.append(temp)


In [169]:
for summary in summaryColumn:
    sentence = summary
    message_embeddings = embed([sentence])
    temp=np.array(message_embeddings[0])
    temp=temp.T
    summary_vector.append(temp)

In [170]:
text_vector=np.array(text_vector)
summary_vector=np.array(summary_vector)

In [171]:
text_vector.shape

(974, 512)

In [172]:
summary_vector.shape

(974, 512)

In [173]:

scalar = StandardScaler()
scaled_data_text = scalar.fit_transform(text_vector) #scaling the data


In [174]:
scalar = StandardScaler()
scaled_data_summary = scalar.fit_transform(summary_vector) #scaling the data

In [175]:
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-1.395093,-1.393648,-1.377238,-1.402761,-0.253108,0.186944,0.341627,1.059945,0.240676,-0.042876,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
1,-1.398045,-1.400842,-1.384250,-1.416864,-0.180434,0.134354,0.265287,1.001807,-0.016483,-0.089443,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
2,-1.388148,-1.398155,-1.362427,-1.409211,-0.536928,0.121484,0.248028,1.102359,0.229683,-0.104037,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
3,-1.390579,-1.364437,-1.357431,-1.385649,-0.649673,0.087785,0.149314,1.049109,0.440741,-0.125033,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
4,-1.368009,-1.394775,-1.373294,-1.389089,-0.221039,0.033378,0.205872,1.283591,0.422234,-0.093246,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,-0.714332,-0.718593,-0.719750,-0.724378,-0.351212,-0.058495,0.034614,-0.074267,-0.136186,-0.170713,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
970,-0.697578,-0.727348,-0.699154,-0.732633,-0.759929,0.036931,0.149686,0.009063,-0.000362,-0.145667,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
971,-0.726746,-0.701344,-0.706429,-0.739169,-0.858032,0.115955,0.141473,-0.159929,-0.279276,-0.202053,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
972,-0.785429,-0.726828,-0.752616,-0.736245,-0.734854,0.051930,0.122974,-0.482416,-0.941740,-0.375894,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898


In [176]:
pca = PCA(n_components = 30)
pca.fit(scaled_data_text)
data_pca_text = pca.transform(scaled_data_text)
data_pca_text

array([[ 1.3779324 ,  1.6211133 ,  0.82854784, ...,  0.82705265,
        -0.3174516 ,  3.5622888 ],
       [ 7.1156216 ,  0.43275154,  4.629278  , ..., -0.894173  ,
        -0.63778114, -0.8356079 ],
       [ 7.9799767 ,  1.1801324 , -0.3386591 , ...,  5.641939  ,
        -0.29106483, -1.3400176 ],
       ...,
       [-2.4728723 , -4.5344896 ,  4.432886  , ..., -1.3145924 ,
        -0.69778895,  0.6351896 ],
       [ 7.440517  , -4.731677  ,  9.186096  , ...,  0.7376535 ,
        -0.5046917 , -3.8108134 ],
       [-5.1189127 , -0.07677031, -0.1855788 , ..., -0.48925593,
         2.9553223 ,  0.8928797 ]], dtype=float32)

In [177]:
pca = PCA(n_components = 30)
pca.fit(scaled_data_summary)
data_pca_summary = pca.transform(scaled_data_summary)
data_pca_summary

array([[-0.237061  , -3.741312  ,  2.9819827 , ..., -1.9641016 ,
        -1.6078358 , -0.8905746 ],
       [-0.237061  , -3.741312  ,  2.9819827 , ..., -1.9641016 ,
        -1.6078358 , -0.8905746 ],
       [-0.2569654 , -3.6267223 ,  5.53924   , ..., -0.3939165 ,
        -0.3198506 ,  0.84708965],
       ...,
       [ 2.4644747 , -8.076461  , -1.3820937 , ...,  1.0344025 ,
        -6.21091   , -2.191347  ],
       [ 2.4644747 , -8.076461  , -1.3820937 , ...,  1.0344025 ,
        -6.21091   , -2.191347  ],
       [-4.352067  ,  0.16857064, -0.6249292 , ..., -0.58264136,
         1.2080173 ,  1.9358413 ]], dtype=float32)

In [178]:
data_pca_text.shape

(974, 30)

In [179]:
data_pca_summary.shape

(974, 30)

In [180]:
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-1.395093,-1.393648,-1.377238,-1.402761,-0.253108,0.186944,0.341627,1.059945,0.240676,-0.042876,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
1,-1.398045,-1.400842,-1.384250,-1.416864,-0.180434,0.134354,0.265287,1.001807,-0.016483,-0.089443,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
2,-1.388148,-1.398155,-1.362427,-1.409211,-0.536928,0.121484,0.248028,1.102359,0.229683,-0.104037,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
3,-1.390579,-1.364437,-1.357431,-1.385649,-0.649673,0.087785,0.149314,1.049109,0.440741,-0.125033,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
4,-1.368009,-1.394775,-1.373294,-1.389089,-0.221039,0.033378,0.205872,1.283591,0.422234,-0.093246,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,-0.714332,-0.718593,-0.719750,-0.724378,-0.351212,-0.058495,0.034614,-0.074267,-0.136186,-0.170713,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
970,-0.697578,-0.727348,-0.699154,-0.732633,-0.759929,0.036931,0.149686,0.009063,-0.000362,-0.145667,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
971,-0.726746,-0.701344,-0.706429,-0.739169,-0.858032,0.115955,0.141473,-0.159929,-0.279276,-0.202053,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
972,-0.785429,-0.726828,-0.752616,-0.736245,-0.734854,0.051930,0.122974,-0.482416,-0.941740,-0.375894,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898


In [181]:
# text_vector_dataframe=pd.DataFrame()
type(data_pca_text)
# df_news  =pd.DataFrame(((x,) for x in data_pca), columns=['title'])
mycolumns=['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8','pca9','pca10','pca11','pca12','pca13','pca14','pca15','pca16','pca17','pca18','pca19','pca20','pca21','pca22','pca23','pca24','pca25','pca26','pca27','pca28','pca29','pca30']
df_news_text=pd.DataFrame(data_pca_text, columns = mycolumns)

In [182]:
# text_vector_dataframe=pd.DataFrame()
type(data_pca_summary)
# df_news  =pd.DataFrame(((x,) for x in data_pca), columns=['title'])
mycolumns=['Spca1','Spca2','Spca3','Spca4','Spca5','Spca6','Spca7','Spca8','Spca9','Spca10','Spca11','Spca12','Spca13','Spca14','Spca15','Spca16','Spca17','Spca18','Spca19','Spca20','Spca21','Spca22','Spca23','Spca24','Spca25','Spca26','Spca27','Spca28','Spca29','Spca30']
df_news_summary=pd.DataFrame(data_pca_summary, columns = mycolumns)

In [183]:
df_news_text

,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,...,pca21,pca22,pca23,pca24,pca25,pca26,pca27,pca28,pca29,pca30
0,1.377932,1.621113,0.828548,-5.151092,2.656592,-1.446503,-6.072556,0.196129,0.358220,0.017776,...,-0.141346,1.728256,0.207593,-1.797425,2.818720,2.158968,0.439347,0.827053,-0.317452,3.562289
1,7.115622,0.432752,4.629278,-5.929930,-0.631628,-3.273161,-2.215599,7.248551,3.305006,0.636301,...,-4.344407,-1.446181,1.529950,-2.639816,-0.463651,-4.332584,2.653840,-0.894173,-0.637781,-0.835608
2,7.979977,1.180132,-0.338659,-1.332992,0.302161,0.035017,1.125854,10.468190,0.026917,1.852854,...,-2.181946,3.506262,-1.731289,3.287947,1.567565,1.215962,2.623448,5.641939,-0.291065,-1.340018
3,-4.992699,-1.467405,-1.481142,-6.944676,-0.953849,0.886891,1.733713,3.412879,-2.172530,1.701016,...,1.834805,2.227690,1.486390,2.995109,2.342221,-0.086258,2.227390,0.168649,-1.505635,-1.859590
4,1.429708,-3.093750,-4.018600,0.306939,-1.641705,-0.405005,2.132976,-2.891635,-1.882236,-3.149434,...,-4.903287,-1.403156,3.964365,4.752428,0.578823,0.681423,-2.467163,-0.019404,-0.346581,1.544068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,-5.086168,-1.129834,0.412271,-1.596922,-1.228268,0.536398,-4.976513,2.109158,1.169629,1.048331,...,3.489858,-0.204522,2.102576,0.803902,1.895770,2.628719,-0.224026,-1.121544,1.854384,1.186308
970,7.477098,-1.437148,-2.014987,-3.452432,-3.221599,-4.005706,2.857090,-0.040639,-2.769229,2.333539,...,0.193567,3.478336,-1.702627,-2.364012,1.131355,4.231129,-1.821960,-2.738722,2.407336,2.217631
971,-2.472872,-4.534490,4.432886,0.314386,-2.269318,-3.147353,0.079204,-2.242477,0.641298,-1.497004,...,4.075458,-0.868174,-1.694066,0.090976,-4.502910,0.986814,-1.329639,-1.314592,-0.697789,0.635190
972,7.440517,-4.731677,9.186096,2.654151,0.001284,-3.571181,0.056479,-2.718827,0.827003,-3.112107,...,1.788279,-3.574491,-3.903393,-2.331117,-6.505133,0.436838,-0.620986,0.737653,-0.504692,-3.810813


In [184]:
df_news_summary

,Spca1,Spca2,Spca3,Spca4,Spca5,Spca6,Spca7,Spca8,Spca9,Spca10,...,Spca21,Spca22,Spca23,Spca24,Spca25,Spca26,Spca27,Spca28,Spca29,Spca30
0,-0.237061,-3.741312,2.981983,2.526859,-7.503465,2.266643,1.578930,3.589519,2.402256,7.353742,...,-2.197240,-4.754534,1.273511,-0.333166,3.354223,1.867052,-1.603935,-1.964102,-1.607836,-0.890575
1,-0.237061,-3.741312,2.981983,2.526859,-7.503465,2.266643,1.578930,3.589519,2.402256,7.353742,...,-2.197240,-4.754534,1.273511,-0.333166,3.354223,1.867052,-1.603935,-1.964102,-1.607836,-0.890575
2,-0.256965,-3.626722,5.539240,1.684384,-3.485335,4.373451,0.402323,-1.254155,-0.535744,10.865088,...,1.061313,3.722758,-0.376442,-0.821632,-0.164640,-1.772278,1.786266,-0.393916,-0.319851,0.847090
3,-0.686974,-4.384386,-6.092415,3.394203,1.769017,-5.422543,-2.521682,-0.673478,-3.069333,1.893385,...,2.439788,-0.963925,0.666494,0.110597,-0.587893,-3.696036,2.327111,4.182050,-3.182351,-0.506240
4,-1.569797,0.477342,1.056403,4.146873,1.198099,-5.058319,2.169601,6.642674,-2.875948,-2.589339,...,-0.415764,-1.177033,-0.165027,-0.050659,0.079380,-1.471988,1.752536,0.745291,3.111745,3.189694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,-0.954344,-5.087415,-7.174898,-0.277118,-0.738791,0.021976,-1.309700,-0.360093,3.558336,-0.143765,...,3.144001,-0.679302,-3.257451,2.200295,-2.875688,3.128153,1.051484,2.995652,3.237191,3.076340
970,3.351356,-4.009085,-0.182468,5.443825,-2.538980,-3.707494,-3.435933,1.295579,-2.069684,0.657242,...,4.557633,-0.503230,-2.424499,-3.956492,0.537309,-2.604434,0.880562,-1.984482,2.059427,5.042179
971,2.464475,-8.076461,-1.382094,3.709399,-0.442495,2.466095,-3.703674,-3.615551,0.725917,-0.112894,...,4.851665,-0.884374,3.992465,2.252100,3.880036,-0.393312,0.941715,1.034402,-6.210910,-2.191347
972,2.464475,-8.076461,-1.382094,3.709399,-0.442495,2.466095,-3.703674,-3.615551,0.725917,-0.112894,...,4.851665,-0.884374,3.992465,2.252100,3.880036,-0.393312,0.941715,1.034402,-6.210910,-2.191347


In [186]:
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-1.395093,-1.393648,-1.377238,-1.402761,-0.253108,0.186944,0.341627,1.059945,0.240676,-0.042876,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
1,-1.398045,-1.400842,-1.384250,-1.416864,-0.180434,0.134354,0.265287,1.001807,-0.016483,-0.089443,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
2,-1.388148,-1.398155,-1.362427,-1.409211,-0.536928,0.121484,0.248028,1.102359,0.229683,-0.104037,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
3,-1.390579,-1.364437,-1.357431,-1.385649,-0.649673,0.087785,0.149314,1.049109,0.440741,-0.125033,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
4,-1.368009,-1.394775,-1.373294,-1.389089,-0.221039,0.033378,0.205872,1.283591,0.422234,-0.093246,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,-0.714332,-0.718593,-0.719750,-0.724378,-0.351212,-0.058495,0.034614,-0.074267,-0.136186,-0.170713,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
970,-0.697578,-0.727348,-0.699154,-0.732633,-0.759929,0.036931,0.149686,0.009063,-0.000362,-0.145667,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
971,-0.726746,-0.701344,-0.706429,-0.739169,-0.858032,0.115955,0.141473,-0.159929,-0.279276,-0.202053,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
972,-0.785429,-0.726828,-0.752616,-0.736245,-0.734854,0.051930,0.122974,-0.482416,-0.941740,-0.375894,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898


In [187]:
df.columns

RangeIndex(start=0, stop=25, step=1)

In [188]:
df=np.array(df)

In [189]:
column=['close', 'open', 'low', 'high', 'volume','evm',  'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
  'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',  'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
   'Free Cash Flow', 'Total Assets',  'Total Liabilities Net Minority Interest', 'Total Equity Gross Minority Interest', 'Total Capitalization']

df=pd.DataFrame(df,columns=column)
    #  'volume', 'y_actual', 'evm',
#    ,'titleSentiment','summarySentiment'

In [190]:
df

,close,open,low,high,volume,evm,force_index,rsi,cci,macd-signal,...,Gross Profit,Normalized EBITDA,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Free Cash Flow,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization
0,-1.395093,-1.393648,-1.377238,-1.402761,-0.253108,0.186944,0.341627,1.059945,0.240676,-0.042876,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
1,-1.398045,-1.400842,-1.384250,-1.416864,-0.180434,0.134354,0.265287,1.001807,-0.016483,-0.089443,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
2,-1.388148,-1.398155,-1.362427,-1.409211,-0.536928,0.121484,0.248028,1.102359,0.229683,-0.104037,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
3,-1.390579,-1.364437,-1.357431,-1.385649,-0.649673,0.087785,0.149314,1.049109,0.440741,-0.125033,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
4,-1.368009,-1.394775,-1.373294,-1.389089,-0.221039,0.033378,0.205872,1.283591,0.422234,-0.093246,...,-1.353713,-1.460026,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.737310,-1.814426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,-0.714332,-0.718593,-0.719750,-0.724378,-0.351212,-0.058495,0.034614,-0.074267,-0.136186,-0.170713,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
970,-0.697578,-0.727348,-0.699154,-0.732633,-0.759929,0.036931,0.149686,0.009063,-0.000362,-0.145667,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
971,-0.726746,-0.701344,-0.706429,-0.739169,-0.858032,0.115955,0.141473,-0.159929,-0.279276,-0.202053,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898
972,-0.785429,-0.726828,-0.752616,-0.736245,-0.734854,0.051930,0.122974,-0.482416,-0.941740,-0.375894,...,1.243641,1.080876,1.130951,0.523018,-1.611869,1.101683,0.839362,1.216949,0.513349,0.412898


In [191]:
df['titleSentiment']=titleSentiment

In [192]:
df['summarySentiment']=summarySentiment

In [193]:
# df=df.drop(['y_actual'],axis='columns')
df.columns

Index(['close', 'open', 'low', 'high', 'volume', 'evm', 'force_index', 'rsi',
       'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3', 'Total Revenue',
       'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',
       'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
       'Free Cash Flow', 'Total Assets',
       'Total Liabilities Net Minority Interest',
       'Total Equity Gross Minority Interest', 'Total Capitalization',
       'titleSentiment', 'summarySentiment'],
      dtype='object')

In [194]:
df.head(5)

,close,open,low,high,volume,evm,force_index,rsi,cci,macd-signal,...,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Free Cash Flow,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,titleSentiment,summarySentiment
0,-1.395093,-1.393648,-1.377238,-1.402761,-0.253108,0.186944,0.341627,1.059945,0.240676,-0.042876,...,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.73731,-1.814426,1,1
1,-1.398045,-1.400842,-1.384250,-1.416864,-0.180434,0.134354,0.265287,1.001807,-0.016483,-0.089443,...,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.73731,-1.814426,1,1
2,-1.388148,-1.398155,-1.362427,-1.409211,-0.536928,0.121484,0.248028,1.102359,0.229683,-0.104037,...,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.73731,-1.814426,1,1
3,-1.390579,-1.364437,-1.357431,-1.385649,-0.649673,0.087785,0.149314,1.049109,0.440741,-0.125033,...,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.73731,-1.814426,1,1
4,-1.368009,-1.394775,-1.373294,-1.389089,-0.221039,0.033378,0.205872,1.283591,0.422234,-0.093246,...,-1.487564,0.609295,1.002006,-1.471732,-1.639139,-1.351827,-1.73731,-1.814426,1,1


In [195]:
newDF=pd.concat([dateColumn,df,df_news_text,df_news_summary],axis=1)


In [196]:
newDF.shape

(974, 88)

In [197]:
newDF.head(5)

,datetime,close,open,low,high,volume,evm,force_index,rsi,cci,...,Spca21,Spca22,Spca23,Spca24,Spca25,Spca26,Spca27,Spca28,Spca29,Spca30
0,2019-02-20,-1.395093,-1.393648,-1.377238,-1.402761,-0.253108,0.186944,0.341627,1.059945,0.240676,...,-2.197240,-4.754534,1.273511,-0.333166,3.354223,1.867052,-1.603935,-1.964102,-1.607836,-0.890575
1,2019-02-21,-1.398045,-1.400842,-1.384250,-1.416864,-0.180434,0.134354,0.265287,1.001807,-0.016483,...,-2.197240,-4.754534,1.273511,-0.333166,3.354223,1.867052,-1.603935,-1.964102,-1.607836,-0.890575
2,2019-02-22,-1.388148,-1.398155,-1.362427,-1.409211,-0.536928,0.121484,0.248028,1.102359,0.229683,...,1.061313,3.722758,-0.376442,-0.821632,-0.164640,-1.772278,1.786266,-0.393916,-0.319851,0.847090
3,2019-02-25,-1.390579,-1.364437,-1.357431,-1.385649,-0.649673,0.087785,0.149314,1.049109,0.440741,...,2.439788,-0.963925,0.666494,0.110597,-0.587893,-3.696036,2.327111,4.182050,-3.182351,-0.506240
4,2019-02-26,-1.368009,-1.394775,-1.373294,-1.389089,-0.221039,0.033378,0.205872,1.283591,0.422234,...,-0.415764,-1.177033,-0.165027,-0.050659,0.079380,-1.471988,1.752536,0.745291,3.111745,3.189694


In [198]:
newDF.columns

Index(['datetime', 'close', 'open', 'low', 'high', 'volume', 'evm',
       'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
       'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',
       'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
       'Free Cash Flow', 'Total Assets',
       'Total Liabilities Net Minority Interest',
       'Total Equity Gross Minority Interest', 'Total Capitalization',
       'titleSentiment', 'summarySentiment', 'pca1', 'pca2', 'pca3', 'pca4',
       'pca5', 'pca6', 'pca7', 'pca8', 'pca9', 'pca10', 'pca11', 'pca12',
       'pca13', 'pca14', 'pca15', 'pca16', 'pca17', 'pca18', 'pca19', 'pca20',
       'pca21', 'pca22', 'pca23', 'pca24', 'pca25', 'pca26', 'pca27', 'pca28',
       'pca29', 'pca30', 'Spca1', 'Spca2', 'Spca3', 'Spca4', 'Spca5', 'Spca6',
       'Spca7', 'Spca8', 'Spca9', 'Spca10', 'Spca11', 'Spca12', 'Spca13',
       'Spca14', 'Spca15', 'Spca16', 'Spca17', 'Spca18', 'Spca19', 'Spca20'

Start applying model after this.

Take data before 2022-9-30 as training and after as testing.


In [199]:
file_train = newDF[(newDF['datetime'] <'2022-01-01')]
file_test = newDF[(newDF['datetime'] >='2022-01-01')]
y_df_train = y_df[(y_df['datetime'] <'2022-01-01')]
y_df_test = y_df[(y_df['datetime'] >='2022-01-01')]
print("train x: ",file_train.shape,"train y: ",y_df_train.shape)
print("test x: ",file_test.shape,"test y: ",y_df_test.shape)



train x:  (724, 88) train y:  (724, 2)
test x:  (250, 88) test y:  (250, 2)


In [200]:
file_train = file_train.drop('datetime', axis=1)
# file_train = file_train.drop('title', axis=1)
file_test = file_test.drop('datetime', axis=1)
y_df_train = y_df_train.drop('datetime', axis=1)
y_df_test = y_df_test.drop('datetime', axis=1)
file_train

,close,open,low,high,volume,evm,force_index,rsi,cci,macd-signal,...,Spca21,Spca22,Spca23,Spca24,Spca25,Spca26,Spca27,Spca28,Spca29,Spca30
0,-1.395093,-1.393648,-1.377238,-1.402761,-0.253108,0.186944,0.341627,1.059945,0.240676,-0.042876,...,-2.197240,-4.754534,1.273511,-0.333166,3.354223,1.867052,-1.603935,-1.964102,-1.607836,-0.890575
1,-1.398045,-1.400842,-1.384250,-1.416864,-0.180434,0.134354,0.265287,1.001807,-0.016483,-0.089443,...,-2.197240,-4.754534,1.273511,-0.333166,3.354223,1.867052,-1.603935,-1.964102,-1.607836,-0.890575
2,-1.388148,-1.398155,-1.362427,-1.409211,-0.536928,0.121484,0.248028,1.102359,0.229683,-0.104037,...,1.061313,3.722758,-0.376442,-0.821632,-0.164640,-1.772278,1.786266,-0.393916,-0.319851,0.847090
3,-1.390579,-1.364437,-1.357431,-1.385649,-0.649673,0.087785,0.149314,1.049109,0.440741,-0.125033,...,2.439788,-0.963925,0.666494,0.110597,-0.587893,-3.696036,2.327111,4.182050,-3.182351,-0.506240
4,-1.368009,-1.394775,-1.373294,-1.389089,-0.221039,0.033378,0.205872,1.283591,0.422234,-0.093246,...,-0.415764,-1.177033,-0.165027,-0.050659,0.079380,-1.471988,1.752536,0.745291,3.111745,3.189694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,1.379168,1.353291,1.393730,1.348005,-0.552280,-1.339519,-0.433754,-0.989710,-0.581353,-2.045468,...,-2.355364,-2.449338,-0.339148,0.348772,2.315236,-0.433321,4.072791,0.484435,-2.684706,-4.634631
720,1.306943,1.420900,1.360952,1.365203,-0.476583,-2.178522,-0.937973,-1.118267,-0.570211,-1.599732,...,2.932383,2.615984,0.117859,-3.185411,1.099507,1.242445,-0.537223,-1.792408,-0.553803,1.479445
721,1.306335,1.338469,1.312136,1.314296,-0.253464,-2.410250,-1.173406,-1.119383,-0.575827,-1.183522,...,2.932383,2.615984,0.117859,-3.185411,1.099507,1.242445,-0.537223,-1.792408,-0.553803,1.479445
722,1.317100,1.314545,1.340707,1.331752,-0.798577,-1.959860,-0.490477,-1.086193,-0.483912,-0.778030,...,0.955317,-1.592365,-3.166720,0.173590,-2.285771,0.139605,-0.815870,0.744774,-0.565867,0.826812


In [201]:
file_test

,close,open,low,high,volume,evm,force_index,rsi,cci,macd-signal,...,Spca21,Spca22,Spca23,Spca24,Spca25,Spca26,Spca27,Spca28,Spca29,Spca30
724,1.263625,1.280567,1.228789,1.297442,-0.029099,-3.060933,-1.546371,-1.199943,-0.582310,-0.287635,...,-0.007714,-1.307538,0.484386,0.941520,6.255443,-0.228977,0.265109,0.380443,-1.357390,2.875904
725,1.173603,1.256731,1.123969,1.205431,0.849449,-1.602152,-1.518909,-1.393025,-1.062570,-0.294602,...,-1.108481,1.255060,-3.428624,-1.680682,-2.810960,7.804828,0.359171,2.423804,-0.293193,-2.992200
726,0.830098,1.006229,0.884619,0.985725,2.657405,-2.195778,-5.160728,-1.975265,-2.470462,-1.034921,...,-0.842480,2.666496,0.404830,0.191419,-4.301915,-0.237862,-1.272467,-0.454182,-0.767846,-0.216913
727,0.827407,0.789532,0.833787,0.878752,0.593845,-1.540763,-1.143842,-1.979118,-2.076618,-1.387336,...,-0.842481,2.666498,0.404829,0.191419,-4.301915,-0.237862,-1.272467,-0.454182,-0.767846,-0.216913
728,0.797718,0.821256,0.796978,0.828103,0.411153,-1.373197,-0.911815,-2.023862,-1.723895,-1.538859,...,-0.714547,0.887547,0.073810,-2.537720,1.234497,-1.427889,-0.468506,-1.260588,3.894638,-2.431152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,-0.714332,-0.718593,-0.719750,-0.724378,-0.351212,-0.058495,0.034614,-0.074267,-0.136186,-0.170713,...,3.144001,-0.679302,-3.257451,2.200295,-2.875688,3.128153,1.051484,2.995652,3.237191,3.076340
970,-0.697578,-0.727348,-0.699154,-0.732633,-0.759929,0.036931,0.149686,0.009063,-0.000362,-0.145667,...,4.557633,-0.503230,-2.424499,-3.956492,0.537309,-2.604434,0.880562,-1.984482,2.059427,5.042179
971,-0.726746,-0.701344,-0.706429,-0.739169,-0.858032,0.115955,0.141473,-0.159929,-0.279276,-0.202053,...,4.851665,-0.884374,3.992465,2.252100,3.880036,-0.393312,0.941715,1.034402,-6.210910,-2.191347
972,-0.785429,-0.726828,-0.752616,-0.736245,-0.734854,0.051930,0.122974,-0.482416,-0.941740,-0.375894,...,4.851665,-0.884374,3.992465,2.252100,3.880036,-0.393312,0.941715,1.034402,-6.210910,-2.191347


In [202]:
y_df_train

,y_actual
0,0.0
1,1.0
2,0.0
3,1.0
4,1.0
...,...
719,0.0
720,0.0
721,1.0
722,0.0


In [203]:
y_df_test

,y_actual
724,0.0
725,0.0
726,0.0
727,0.0
728,1.0
...,...
969,1.0
970,0.0
971,0.0
972,1.0


In [205]:
print(y_df_test)

     y_actual
724       0.0
725       0.0
726       0.0
727       0.0
728       1.0
..        ...
969       1.0
970       0.0
971       0.0
972       1.0
973       0.0

[250 rows x 1 columns]


In [206]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report




baseline_LR = LogisticRegression(C = 1e10, tol=0.000000001, max_iter=100000)
baseline_LR.fit(file_train, y_df_train)
    # print('Coefficients: \n', baseline_LR.coef_)
print("training accracy: %.4f" % baseline_LR.score(file_train, y_df_train))
print("test accuracy: %.4f" % baseline_LR.score(file_test, y_df_test))
y_fit_LR = baseline_LR.predict(file_test)
# print(y_fit_LR)
# print(y_df_test)
print('Accuracy:', accuracy_score(y_df_test, y_fit_LR))
print('ROC AUC Score:', roc_auc_score(y_df_test, y_fit_LR))
print('F1 score:', f1_score(y_df_test, y_fit_LR))
print()
print('\n clasification report:\n', classification_report(y_df_test, y_fit_LR))
print('\n confussion matrix:\n',confusion_matrix(y_df_test, y_fit_LR))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training accracy: 0.6671
test accuracy: 0.4680
Accuracy: 0.468
ROC AUC Score: 0.48520329387545036
F1 score: 0.5581395348837209


 clasification report:
               precision    recall  f1-score   support

         0.0       0.51      0.25      0.33       134
         1.0       0.45      0.72      0.56       116

    accuracy                           0.47       250
   macro avg       0.48      0.49      0.44       250
weighted avg       0.48      0.47      0.44       250


 confussion matrix:
 [[ 33 101]
 [ 32  84]]


In [207]:
# Fit and report Logistic Regression
baseline_LR = LogisticRegression(C = 1e1, tol=0.000000001, max_iter=100000)
baseline_LR.fit(file_train, y_df_train)
# Training acc
pred = baseline_LR.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the Logistic Regression is ', accuracy)
# Test acc
y_hat = baseline_LR.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the Logistic Regression is ', test_accuracy)
# Conf matrix
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The training accuracy of the Logistic Regression is  0.6685082872928176
The test accuracy of the Logistic Regression is  0.492

 Training clasification report:
               precision    recall  f1-score   support

         0.0       0.65      0.52      0.58       316
         1.0       0.68      0.78      0.73       408

    accuracy                           0.67       724
   macro avg       0.66      0.65      0.65       724
weighted avg       0.67      0.67      0.66       724


 confusion matrix:
 [[165 151]
 [ 89 319]]

 Test clasification report:
               precision    recall  f1-score   support

         0.0       0.58      0.19      0.28       134
         1.0       0.47      0.84      0.61       116

    accuracy                           0.49       250
   macro avg       0.53      0.52      0.44       250
weighted avg       0.53      0.49      0.43       250


 confusion matrix:
 [[ 25 109]
 [ 18  98]]


In [208]:
from sklearn import svm

In [209]:
# Fit and report SVM
    # c = 0.5
svm_rbf = svm.SVC(C=0.5, kernel='rbf',max_iter=50000)
svm_rbf.fit(file_train, y_df_train)
# Training acc
pred = svm_rbf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the SVM is ', accuracy)
# Test acc
y_hat = svm_rbf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the SVM is ', test_accuracy)
# Conf matrix
print('\n Training classification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test classification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The training accuracy of the SVM is  0.6546961325966851
The test accuracy of the SVM is  0.456

 Training classification report:
               precision    recall  f1-score   support

         0.0       0.84      0.26      0.40       316
         1.0       0.63      0.96      0.76       408

    accuracy                           0.65       724
   macro avg       0.73      0.61      0.58       724
weighted avg       0.72      0.65      0.60       724


 confusion matrix:
 [[ 82 234]
 [ 16 392]]

 Test classification report:
               precision    recall  f1-score   support

         0.0       0.43      0.04      0.08       134
         1.0       0.46      0.93      0.61       116

    accuracy                           0.46       250
   macro avg       0.44      0.49      0.35       250
weighted avg       0.44      0.46      0.33       250


 confusion matrix:
 [[  6 128]
 [  8 108]]


In [210]:
# Fit and report SVM
    # c = 1e10
svm_rbf = svm.SVC(C=1e10, kernel='rbf',max_iter=100000)
svm_rbf.fit(file_train, y_df_train)
# Training acc
pred = svm_rbf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the SVM is ', accuracy)
# Test acc
y_hat = svm_rbf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the SVM is ', test_accuracy)
# Conf matrix
print('\n Training classification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test classification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The training accuracy of the SVM is  1.0
The test accuracy of the SVM is  0.484

 Training classification report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       316
         1.0       1.00      1.00      1.00       408

    accuracy                           1.00       724
   macro avg       1.00      1.00      1.00       724
weighted avg       1.00      1.00      1.00       724


 confusion matrix:
 [[316   0]
 [  0 408]]

 Test classification report:
               precision    recall  f1-score   support

         0.0       0.67      0.07      0.13       134
         1.0       0.47      0.96      0.63       116

    accuracy                           0.48       250
   macro avg       0.57      0.52      0.38       250
weighted avg       0.58      0.48      0.37       250


 confusion matrix:
 [[ 10 124]
 [  5 111]]


In [211]:
# Fit and report SVM
    # c = 1e1
svm_rbf = svm.SVC(C=1e1, kernel='rbf',max_iter=100000)
svm_rbf.fit(file_train, y_df_train)
# Training acc
pred = svm_rbf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the SVM is ', accuracy)
# Test acc
y_hat = svm_rbf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the SVM is ', test_accuracy)
# Conf matrix
print('\n Training classification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test classification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The training accuracy of the SVM is  0.9406077348066298
The test accuracy of the SVM is  0.568

 Training classification report:
               precision    recall  f1-score   support

         0.0       0.96      0.91      0.93       316
         1.0       0.93      0.97      0.95       408

    accuracy                           0.94       724
   macro avg       0.94      0.94      0.94       724
weighted avg       0.94      0.94      0.94       724


 confusion matrix:
 [[286  30]
 [ 13 395]]

 Test classification report:
               precision    recall  f1-score   support

         0.0       0.62      0.49      0.55       134
         1.0       0.53      0.66      0.59       116

    accuracy                           0.57       250
   macro avg       0.58      0.57      0.57       250
weighted avg       0.58      0.57      0.57       250


 confusion matrix:
 [[65 69]
 [39 77]]


In [212]:
from sklearn.ensemble import RandomForestClassifier

In [213]:
# Fit Random Forest
rf = RandomForestClassifier(n_estimators=10, oob_score=True, random_state=0)
rf.fit(file_train, y_df_train)
# Training acc
pred = rf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the Random Forest is ', accuracy)
# Test acc
y_hat = rf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the Random Forest is ', test_accuracy)
# Conf matrix
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

The training accuracy of the Random Forest is  0.9917127071823204
The test accuracy of the Random Forest is  0.464

 Training clasification report:
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       316
         1.0       0.99      0.99      0.99       408

    accuracy                           0.99       724
   macro avg       0.99      0.99      0.99       724
weighted avg       0.99      0.99      0.99       724


 confusion matrix:
 [[313   3]
 [  3 405]]

 Test clasification report:
               precision    recall  f1-score   support

         0.0       0.50      0.43      0.46       134
         1.0       0.43      0.50      0.46       116

    accuracy                           0.46       250
   macro avg       0.47      0.47      0.46       250
weighted avg       0.47      0.46      0.46       250


 confusion matrix:
 [[58 76]
 [58 58]]


<ipython-input-213-30f767e98b6f>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(file_train, y_df_train)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


In [214]:
# Fit Random Forest
rf = RandomForestClassifier(n_estimators=250, oob_score=True, random_state=123456)
rf.fit(file_train, y_df_train)
# Training acc
pred = rf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the Random Forest is ', accuracy)
# Test acc
y_hat = rf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the Random Forest is ', test_accuracy)
# Conf matrix
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

<ipython-input-214-f7ee8b367a03>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(file_train, y_df_train)


The training accuracy of the Random Forest is  1.0
The test accuracy of the Random Forest is  0.5

 Training clasification report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       316
         1.0       1.00      1.00      1.00       408

    accuracy                           1.00       724
   macro avg       1.00      1.00      1.00       724
weighted avg       1.00      1.00      1.00       724


 confusion matrix:
 [[316   0]
 [  0 408]]

 Test clasification report:
               precision    recall  f1-score   support

         0.0       0.70      0.12      0.20       134
         1.0       0.48      0.94      0.64       116

    accuracy                           0.50       250
   macro avg       0.59      0.53      0.42       250
weighted avg       0.60      0.50      0.40       250


 confusion matrix:
 [[ 16 118]
 [  7 109]]


In [215]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import RMSprop,Adam
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

In [216]:
m1, n1 = file_train.shape
x_train = np.array(file_train).reshape(m1, n1, 1)
m2, n2 = file_test.shape
x_test= np.array(file_test).reshape(m2, n2, 1)

In [217]:
train_confusion = np.zeros((2, 2))
test_confusion = np.zeros((2, 2))

batch_size = 128
epochs = 80
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
#model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=x_train.shape[1:3]))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
print(x_train.shape)
# print(filename)
# if x_train.shape[0] == 0:
#     continue

# Train model
history = model.fit(x_train, y_df_train,batch_size=batch_size,   epochs=epochs,verbose=1)
                
             

# Training report
train_eval = model.evaluate(x_train, y_df_train, verbose=0)
print('Training loss:', train_eval[0])
print('Training accuracy of CNN:', train_eval[1])


pred = model.predict(x_train)
pred = (pred > 0.5)
train_conf = confusion_matrix(y_df_train, pred)
train_confusion += train_conf
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print(train_conf)




# Test report
test_eval = model.evaluate(x_test, y_df_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy of CNN:',test_eval[1])


y_hat = model.predict(x_test)
y_hat = (y_hat > 0.5)
test_conf = confusion_matrix(y_df_test, y_hat)
test_confusion += test_conf
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print(test_conf)
    

(724, 87, 1)
Epoch 1/80
6/6 [==============================] - 1s 23ms/step - loss: 0.7259 - accuracy: 0.5414
Epoch 2/80
6/6 [==============================] - 0s 25ms/step - loss: 0.6923 - accuracy: 0.5345
Epoch 3/80
6/6 [==============================] - 0s 27ms/step - loss: 0.6908 - accuracy: 0.5594
Epoch 4/80
6/6 [==============================] - 0s 25ms/step - loss: 0.6914 - accuracy: 0.5331
Epoch 5/80
6/6 [==============================] - 0s 28ms/step - loss: 0.6847 - accuracy: 0.5511
Epoch 6/80
6/6 [==============================] - 0s 23ms/step - loss: 0.6805 - accuracy: 0.5663
Epoch 7/80
6/6 [==============================] - 0s 25ms/step - loss: 0.6901 - accuracy: 0.5373
Epoch 8/80
6/6 [==============================] - 0s 24ms/step - loss: 0.6918 - accuracy: 0.5552
Epoch 9/80
6/6 [==============================] - 0s 29ms/step - loss: 0.6841 - accuracy: 0.5608
Epoch 10/80
6/6 [==============================] - 0s 25ms/step - loss: 0.6816 - accuracy: 0.5732
Epoch 11/80
6/6 

In [218]:
# train_confusion = np.zeros((2, 2))
# test_confusion = np.zeros((2, 2))

batch_size = 32
epochs = 200
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))

model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
#model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=x_train.shape[1:3]))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
print(x_train.shape)


# Train model
history = model.fit(x_train, y_df_train,batch_size=batch_size,   epochs=epochs,verbose=1)
                
             
# Training report
train_eval = model.evaluate(x_train, y_df_train, verbose=0)
print('Training loss:', train_eval[0])
print('Training accuracy of CNN:', train_eval[1])


pred = model.predict(x_train)
pred = (pred > 0.5)
train_conf = confusion_matrix(y_df_train, pred)
train_confusion += train_conf
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print(train_conf)




# Test report
test_eval = model.evaluate(x_test, y_df_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy of CNN:',test_eval[1])


y_hat = model.predict(x_test)
y_hat = (y_hat > 0.5)
test_conf = confusion_matrix(y_df_test, y_hat)
test_confusion += test_conf
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print(test_conf)
    

(724, 87, 1)
Epoch 1/200
23/23 [==============================] - 2s 21ms/step - loss: 0.6983 - accuracy: 0.5304
Epoch 2/200
23/23 [==============================] - 1s 29ms/step - loss: 0.6942 - accuracy: 0.5235
Epoch 3/200
23/23 [==============================] - 1s 27ms/step - loss: 0.6848 - accuracy: 0.5552
Epoch 4/200
23/23 [==============================] - 1s 22ms/step - loss: 0.6876 - accuracy: 0.5497
Epoch 5/200
23/23 [==============================] - 0s 21ms/step - loss: 0.6930 - accuracy: 0.5249
Epoch 6/200
23/23 [==============================] - 1s 22ms/step - loss: 0.6842 - accuracy: 0.5691
Epoch 7/200
23/23 [==============================] - 0s 20ms/step - loss: 0.6814 - accuracy: 0.5635
Epoch 8/200
23/23 [==============================] - 0s 21ms/step - loss: 0.6818 - accuracy: 0.5663
Epoch 9/200
23/23 [==============================] - 0s 19ms/step - loss: 0.6811 - accuracy: 0.5663
Epoch 10/200
23/23 [==============================] - 0s 19ms/step - loss: 0.6844 - acc

In [219]:
# train_confusion = np.zeros((2, 2))
# test_confusion = np.zeros((2, 2))

# batch_size = 16
# epochs = 500
# model = Sequential()
# model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

# #model.add(Conv1D(128, 3, activation='relu'))
# model.add(MaxPooling1D(3))

# #model.add(Conv1D(128, 3, activation='relu'))
# model.add(Conv1D(128, 3, activation='relu'))
# model.add(GlobalAveragePooling1D())
# model.add(Dropout(0.4))
# model.add(Dense(1, activation='sigmoid'))
# """
# #model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=x_train.shape[1:3]))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))
# """
# model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
# print(x_train.shape)


# # Train model
# history = model.fit(x_train, y_df_train,batch_size=batch_size,   epochs=epochs,verbose=1)
                


batch_size = 64
epochs = 200
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))

# model.add(Conv1D(256, 3, activation='relu'))
# model.add(MaxPooling1D(3))

model.add(Conv1D(512, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.9))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.9))

model.add(Dense(1, activation='relu'))

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
# print(x_train.shape)

# Train model
history = model.fit(x_train, y_df_train,batch_size=batch_size, epochs=epochs,verbose=1)


    # Training report
train_eval = model.evaluate(x_train, y_df_train, verbose=0)
print('Training loss:', train_eval[0])
print('Training accuracy of CNN:', train_eval[1])


pred = model.predict(x_train)
pred = (pred > 0.5)
train_conf = confusion_matrix(y_df_train, pred)
train_confusion += train_conf
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print(train_conf)




# Test report
test_eval = model.evaluate(x_test, y_df_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy of CNN:',test_eval[1])


y_hat = model.predict(x_test)
y_hat = (y_hat > 0.5)
test_conf = confusion_matrix(y_df_test, y_hat)
test_confusion += test_conf
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print(test_conf)
    

Epoch 1/200
12/12 [==============================] - 3s 149ms/step - loss: 5.7880 - accuracy: 0.4862
Epoch 2/200
12/12 [==============================] - 1s 117ms/step - loss: 6.2799 - accuracy: 0.4931
Epoch 3/200
12/12 [==============================] - 1s 80ms/step - loss: 6.3936 - accuracy: 0.5055
Epoch 4/200
12/12 [==============================] - 1s 117ms/step - loss: 6.4184 - accuracy: 0.5138
Epoch 5/200
12/12 [==============================] - 2s 145ms/step - loss: 6.4450 - accuracy: 0.4903
Epoch 6/200
12/12 [==============================] - 2s 127ms/step - loss: 6.2315 - accuracy: 0.5166
Epoch 7/200
12/12 [==============================] - 1s 80ms/step - loss: 6.2820 - accuracy: 0.5249
Epoch 8/200
12/12 [==============================] - 1s 83ms/step - loss: 6.5785 - accuracy: 0.5110
Epoch 9/200
12/12 [==============================] - 1s 81ms/step - loss: 6.3218 - accuracy: 0.5221
Epoch 10/200
12/12 [==============================] - 1s 95ms/step - loss: 7.0269 - accuracy: 0

ADABOOST

In [222]:
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=200, learning_rate=0.01)# 200 0.01
                        
# Train Adaboost Classifer
model = abc.fit(file_train, y_df_train)

pred = model.predict(file_train)

#Predict the response for test dataset
y_pred = model.predict(file_test)

# Model Accuracy, how often is the classifier correct?
print("Testing Accuracy:",metrics.accuracy_score(y_df_train, pred))
print("Testing Accuracy:",metrics.accuracy_score(y_df_test, y_pred))


print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test clasification report:\n', classification_report(y_df_test, y_pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_pred))




/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Testing Accuracy: 0.6284530386740331
Testing Accuracy: 0.46

 Training clasification report:
               precision    recall  f1-score   support

         0.0       0.83      0.19      0.30       316
         1.0       0.61      0.97      0.75       408

    accuracy                           0.63       724
   macro avg       0.72      0.58      0.53       724
weighted avg       0.70      0.63      0.55       724


 confusion matrix:
 [[ 59 257]
 [ 12 396]]

 Test clasification report:
               precision    recall  f1-score   support

         0.0       0.43      0.02      0.04       134
         1.0       0.46      0.97      0.62       116

    accuracy                           0.46       250
   macro avg       0.44      0.49      0.33       250
weighted avg       0.44      0.46      0.31       250


 confusion matrix:
 [[  3 131]
 [  4 112]]


In [223]:
# Load libraries
from sklearn.ensemble import AdaBoostClassifier

# Import Support Vector Classifier
from sklearn.svm import SVC
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
svc=SVC(probability=True, kernel='rbf')

# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=200, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model = abc.fit(file_train, y_df_train)

pred = model.predict(file_train)

#Predict the response for test dataset
y_pred = model.predict(file_test)


# Model Accuracy, how often is the classifier correct?
print("Training Accuracy:",metrics.accuracy_score(y_df_train, pred))

print("Testing Accuracy:",metrics.accuracy_score(y_df_test, y_pred))




print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test clasification report:\n', classification_report(y_df_test, y_pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_pred))



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Training Accuracy: 0.5593922651933702
Testing Accuracy: 0.464

 Training clasification report:
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       316
         1.0       0.56      0.99      0.72       408

    accuracy                           0.56       724
   macro avg       0.28      0.50      0.36       724
weighted avg       0.32      0.56      0.40       724


 confusion matrix:
 [[  0 316]
 [  3 405]]

 Test clasification report:
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       134
         1.0       0.46      1.00      0.63       116

    accuracy                           0.46       250
   macro avg       0.23      0.50      0.32       250
weighted avg       0.22      0.46      0.29       250


 confusion matrix:
 [[  0 134]
 [  0 116]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
